In [190]:
import sys
import os
import json
import pandas as pd
import numpy as np
import json
from collections import Counter

base_dir = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(os.path.join(base_dir, "Data_Augmentation"))

from llm import llm_response

In [237]:
# print(gpt_df_with_answerability.loc[7, "answer"])
# print(gpt_df_with_answerability.loc[7, "complete_answer"])
# print(gpt_df_with_answerability.loc[7, "generated_follow_up"])
# print(gpt_df_with_answerability.loc[7, "generated_follow_up_answerability"])
full_df = pd.read_json('informativeness_output/full_df_with_answerability.json')
org_df = pd.read_json('informativeness_output/org_df_with_answerability.json')
gpt_df = pd.read_json('informativeness_output/gpt_df_with_answerability.json')

ERROR_MSG = "LLM failed to generate a response"

idx_with_incorrect_eval = []
example_generated_follow_up = []

def analyze_informativeness(df):
    counter = Counter()
    
    for idx, data in df.iterrows():
        follow_up_answerability = data["generated_follow_up_answerability"].replace("“", '"').replace("”", '"').replace("'", '"')

        if follow_up_answerability == ERROR_MSG: continue

        # print(f"idx: {data["id"]} : {data["generated_follow_up_answerability"]}\n")
        nested_list = json.loads(follow_up_answerability)

        if len(nested_list) != len(data["generated_follow_up"]):
            idx_with_incorrect_eval.append(idx)
            example_generated_follow_up.append(data)

        if ('Complete Answer','Original Answer') in nested_list:
            print(idx)

        counter.update(tuple(sublist) for sublist in nested_list)

    return counter

full_result = analyze_informativeness(full_df)
org_result = analyze_informativeness(org_df)
gpt_result = analyze_informativeness(gpt_df)
print(idx_with_incorrect_eval)
    

[]


In [227]:
# for example in example_generated_follow_up:
#     print(example)

In [224]:
# with open("../../Data_Augmentation/test_complete_answers.json", "r") as f:
# # Data_Augmentation/test.json 
# # with open("cleaned_train.json", "r") as f:
#     complete_answer_data = pd.DataFrame(json.load(f))

# assert (len(full_df) == len(complete_answer_data))

# df_with_errors = pd.DataFrame(example_generated_follow_up)
# df_with_errors = df_with_errors.reset_index()
# df_with_errors

In [225]:
# # # Uncomment the code block below to run the api call again (it takes 2.5 Hours to run the entire dataset)
# # Call GPT API for each model's (Full, Org, GPT) OA/CA/FUQs Pairs
# index_raises_error = []
# num_data = len(full_df)

# ERROR_MESSAGE = 'LLM failed to generate a response'

# def get_llm_response(i, df, ca_df):
#     OA = df.loc[i, 'answer']
#     CA = ca_df.loc[i, "complete_answer"]
#     FUQ = df.loc[i, "generated_follow_up"]

#     question = f"Please tell me whether each of the questions in 'Follow Up Questions' is answerable by either/both the 'Original Answer' and the 'Complete Answer'. Original Answer: {OA}, Complete Answer: {CA}, Follow Up Questions: {FUQ}"
#     prompt = """
#         You must strictly follow these instructions:

#         - Given an "Original Answer" and a "Complete Answer," evaluate a list of "Follow-Up Questions."
#         - For each Follow-Up Question, determine whether it can be answered by:
#             - The "Original Answer" only → Return ["Original Answer"]
#             - The "Complete Answer" only → Return ["Complete Answer"]
#             - Both answers → Return ["Original Answer", "Complete Answer"]
#             - Neither answer → Return []

#         ### **Criteria for Answerability (Assume Answerability Unless Completely Unrelated):**
#         A Follow-Up Question (FUQ) is considered **answerable** if it contains **any relevant information** that connects to the answer in some way.  
#         The answer does **not** need to be explicit, as long as it provides **some useful context** that **relates to** or **touches on** the FUQ's topic.

#         1. **Direct Coverage** ✅  
#         - The answer explicitly provides the required information.  
#         - No inference is needed.  
#         - Example:  
#             - FUQ: "What is blackbody radiation?"  
#             - OA: "Blackbody radiation occurs when hot objects emit their own light."  
#             - ✅ Answerable by OA.  

#         2. **Loose Inferability (Minimal Context Needed) ✅**  
#         - The answer **does not directly answer** the FUQ but provides **some level of relevant context**, even if vague.  
#         - If **some background knowledge** allows for a connection, it should be marked as answerable.  
#         - Example:  
#             - FUQ: "How do blackbody radiation and white light differ in intensity?"  
#             - OA: "Sufficiently hot objects produce their own light, a process called blackbody radiation."  
#             - ✅ Answerable by OA (since blackbody radiation involves emitted light, a connection to intensity can be made).  

#         3. **Any Shared Concept Counts** ✅  
#         - If the FUQ **mentions a topic** that appears in the OA or CA, it should be considered answerable.  
#         - Even if the answer does **not fully explain** the FUQ, **any overlap** makes it answerable.  
#         - Example:  
#             - FUQ: "What are some examples of high-energy emitting materials?"  
#             - OA: "Torches, incandescent bulbs, and the sun emit light through blackbody radiation."  
#             - ✅ Answerable by OA (mention of blackbody radiation implies high-energy emission).  

#         4. **Very Flexible Scope** ✅  
#         - The FUQ **should be marked as answerable unless it is completely unrelated**.  
#         - If **even a minor part** of the FUQ is **somewhat addressed**, it should be considered answerable.  
#         - Example:  
#             - FUQ: "How does thermal decomposition contribute to the emission of white light?"  
#             - OA: "Sufficiently hot objects produce their own light."  
#             - ✅ Answerable by OA (mention of heat and light emission allows for an inference).  

#         5. **Extreme Leniency for Partial Answers** ✅  
#         - A FUQ **does not have to be fully answered** to be considered answerable.  
#         - Even if the answer **only hints at the topic** or **briefly addresses it**, it should be marked as answerable.  
#         - Example:  
#             - FUQ: "How does the wavelength of light affect its energy?"  
#             - OA: "Sufficiently hot objects emit their own light."  
#             - ✅ Answerable by OA (even though the full explanation isn't there, it touches on emitted light). 

#         ### **STRICT OUTPUT FORMAT RULES (DO NOT VIOLATE)**
#         1. **The output must be a single-line string list.**  
#         - Example output:  
#             "[[], ["Complete Answer"], [], ["Original Answer", "Complete Answer"], ["Original Answer"]]"  
#         - 🚫 Do **not** return separate lines.  
#         - 🚫 Do **not** return explanations, summaries, or any extra text.  

#         2. **Every Follow-Up Question must have exactly one corresponding entry.**  
#         - The length of the output list **must match** the number of FUQs provided.  

#         3. **Do NOT include escape characters ('\') or newline characters ('\n').**  
#         - The output should be clean JSON without unnecessary formatting issues.  

#         4. **The response must be a SINGLE list with elements separated by comma (',') wrapping all results.**  
#         - 🚫 Incorrect Example:  
#             "["Original Answer", "Complete Answer"], ["Original Answer", "Complete Answer"], ["Complete Answer"], []"
#         - ✅ Correct Example:  
#             "[["Original Answer", "Complete Answer"], ["Original Answer", "Complete Answer"], ["Complete Answer"], []]"

#         5. **Do NOT add any extra text.**  
#         - **ONLY return the structured list** and nothing else.  

#         6. **POST-PROCESSING:**
#         - **After generating the response, apply this replacement rule to ensure clean formatting:**  
#             response = response.replace("“", '"').replace("”", '"').replace("'", '"').replace("\n", "").replace("\\", "")
#         - This ensures the output is **valid JSON format** without incorrect quotation marks.

#         ### **Input Data:**
#         Original Answer: {OA}  
#         Complete Answer: {CA}  
#         Follow-Up Questions: {FUQ}  

#         Now, analyze the given Follow-Up Questions and return ONLY the correctly structured list and nothing more:
#     """

#     return llm_response(prompt, question)

# def save_response_to_df(i, df, response):
#     df.loc[i, "generated_follow_up_answerability"] = response

# for i in range(len(df_with_errors)):
# # for i in range(5):
#     # Full Dataset
#     response = get_llm_response(i, df_with_errors, complete_answer_data)

#     if response == ERROR_MESSAGE:
#         index_raises_error.append(i)

#     save_response_to_df(i, df_with_errors, response)

In [226]:
# # fixing rows that have incorrectly generated answerability data
# for row in df_with_errors.iterrows():
#     print(row[1].generated_follow_up)
#     print(row[1].generated_follow_up_answerability.replace("\"", '\''))

In [238]:
full_result

Counter({('Complete Answer',): 712,
         (): 676,
         ('Original Answer', 'Complete Answer'): 397,
         ('Original Answer',): 255})

In [239]:
org_result

Counter({('Original Answer', 'Complete Answer'): 610,
         (): 592,
         ('Original Answer',): 582,
         ('Complete Answer',): 542,
         ('Complete Answer', 'Original Answer'): 1})

In [241]:
# Add the count of ('Complete Answer', 'Original Answer') to ('Original Answer', 'Complete Answer')
org_result[('Original Answer', 'Complete Answer')] += org_result[('Complete Answer', 'Original Answer')]

# Remove the ('Complete Answer', 'Original Answer') entry
del org_result[('Complete Answer', 'Original Answer')]

org_result

Counter({('Original Answer', 'Complete Answer'): 611,
         (): 592,
         ('Original Answer',): 582,
         ('Complete Answer',): 542})

In [240]:
gpt_result

Counter({('Complete Answer',): 674,
         (): 624,
         ('Original Answer', 'Complete Answer'): 400,
         ('Original Answer',): 179})

In [248]:
template_table = pd.DataFrame([
    ["Generated FollowupQ", "Answered by CA", "Unanswered by CA"],
    ["Answered by OA", "No new information", "Inappropriate CA"],
    ["Unanswered by OA", "New Information", "Unrelated followupQ"]
])

def display_result(df, table):
    count = 0

    for value in df.values():
        count += value

    for key, value in df.items():
        percentage = round((value / count), 2)
        match key:
            case ():
                table.iloc[2,2] = percentage
            case ('Complete Answer',):
                table.iloc[2,1] = percentage
            case ('Original Answer',):
                table.iloc[1,2] = percentage
            case ('Original Answer', 'Complete Answer'):
                table.iloc[1,1] = percentage
    
    return table

full_result_table = display_result(full_result, template_table.copy())
org_result_table = display_result(org_result, template_table.copy())
gpt_result_table = display_result(gpt_result, template_table.copy())

In [249]:
full_result_table

,0,1,2
0,Generated FollowupQ,Answered by CA,Unanswered by CA
1,Answered by OA,0.19,0.12
2,Unanswered by OA,0.35,0.33


In [250]:
org_result_table

,0,1,2
0,Generated FollowupQ,Answered by CA,Unanswered by CA
1,Answered by OA,0.26,0.25
2,Unanswered by OA,0.23,0.25


In [251]:
gpt_result_table

,0,1,2
0,Generated FollowupQ,Answered by CA,Unanswered by CA
1,Answered by OA,0.21,0.1
2,Unanswered by OA,0.36,0.33
